# Туториал по квантованию с помощью интструментов PyTorch

В этом туториале рассмотрим, как выполнить статическое квантование после обучения, а также разберем два более продвинутых метода — квантование по каналам и обучение с учетом квантования (QAT - Quantization Aware Training) — для дальнейшего повышения точности модели. <br> 

Рассмотрим все на примере задачи MNIST с помощью простой архитектуры LeNet. 


Данный туториал достаточно мимиалистичен для теории и более глубоких объяснений того, что на самом деле происходит, я бы рекомендовал ознакомиться с: [Квантование глубоких сверточных сетей для эффективного инференса
](https://arxiv.org/abs/1806.08342).

Туториал в значительной степени адаптирован из: https://pytorch.org/tutorials/advanced/static_quantization_tutorial.html

### Начало 

Перед началом квантования мы импортируем набор данных MNIST и обучим простую сверточную нейронную сеть (CNN) для задачи классификации.

In [2]:
#!pip3 install torch==1.5.0 torchvision==1.6.0
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import DataLoader
import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

Загрузим обучающие и тестовые данные, применим нормализацию и преобразования.

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=16, pin_memory=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=16, pin_memory=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:07<00:00, 1372179.48it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 209783.48it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1740526.46it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1633836.09it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Определим некоторые вспомогательные функции и классы, которые помогут нам отслеживать статистики и качество работы модели.

In [5]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def accuracy(output, target):
    """ Computes the top 1 accuracy """
    with torch.no_grad():
        batch_size = target.size(0)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        correct_one = correct[:1].view(-1).float().sum(0, keepdim=True)
        return correct_one.mul_(100.0 / batch_size).item()

def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def load_model(quantized_model, model):
    """ Loads in the weights into an object meant for quantization """
    state_dict = model.state_dict()
    model = model.to('cpu')
    quantized_model.load_state_dict(state_dict)

def fuse_modules(model):
    """ Fuse together convolutions/linear layers and ReLU """
    torch.quantization.fuse_modules(model, [['conv1', 'relu1'],
                                            ['conv2', 'relu2'],
                                            ['fc1', 'relu3'],
                                            ['fc2', 'relu4']], inplace=True)

Определим простую сверточную нейронную сеть, для классификации изображений MNIST. <br>

Уделите внимание на части  'QuantStub()' и 'DeQuantStub()' это служебные функции для того что бы определить где у нас начинается, а где заканчивается квантование.


In [12]:
class Net(nn.Module):
    def __init__(self, q = False):
        # By turning on Q we can turn on/off the quantization
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, bias=False)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, bias=False)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256, 120, bias=False)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84, bias=False)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10, bias=False)
        self.q = q
        if q:
          self.quant = QuantStub()
          self.dequant = DeQuantStub()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.q:
          x = self.quant(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        # Be careful to use reshape here instead of view
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        if self.q:
          x = self.dequant(x)
        return x

In [13]:
net = Net(q=False).cuda()
print_size_of_model(net)

Size (MB): 0.179057


Обучим эту модель на обучающем наборе данных (это может занять несколько минут).

In [10]:
def train(model: nn.Module, dataloader: DataLoader, cuda=False, q=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()
    for epoch in range(20):  # loop over the dataset multiple times

        running_loss = AverageMeter('loss')
        acc = AverageMeter('train_acc')
        for i, data in enumerate(dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            if epoch>=3 and q:
              model.apply(torch.quantization.disable_observer)

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss.update(loss.item(), outputs.shape[0])
            acc.update(accuracy(outputs, labels), outputs.shape[0])
            if i % 100 == 0:    # print every 100 mini-batches
                print('[%d, %5d] ' %
                    (epoch + 1, i + 1), running_loss, acc)
    print('Finished Training')


def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data

            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [14]:
train(net, trainloader, cuda=True)

[1,     1]  loss 2.303799 (2.303799) train_acc 3.125000 (3.125000)
[1,   101]  loss 2.293097 (2.302622) train_acc 20.312500 (7.967203)
[1,   201]  loss 2.288037 (2.298191) train_acc 29.687500 (14.995336)
[1,   301]  loss 2.281747 (2.293290) train_acc 32.812500 (20.276163)
[1,   401]  loss 2.243036 (2.285806) train_acc 53.125000 (25.027276)
[1,   501]  loss 2.197001 (2.273354) train_acc 43.750000 (28.686377)
[1,   601]  loss 1.899132 (2.245754) train_acc 57.812500 (31.863561)
[1,   701]  loss 1.420953 (2.168343) train_acc 68.750000 (35.362429)
[1,   801]  loss 0.756028 (2.025936) train_acc 79.687500 (39.965668)
[1,   901]  loss 0.594413 (1.869885) train_acc 82.812500 (44.681257)
[2,     1]  loss 0.679886 (0.679886) train_acc 76.562500 (76.562500)
[2,   101]  loss 0.301674 (0.434280) train_acc 90.625000 (86.772896)
[2,   201]  loss 0.290412 (0.398816) train_acc 89.062500 (88.184080)
[2,   301]  loss 0.416183 (0.365731) train_acc 90.625000 (89.135174)
[2,   401]  loss 0.417450 (0.345215) 

Теперь, когда CNN обучена, давайте проверим ее на нашем тестовом наборе данных.

In [15]:
score = test(net, testloader, cuda=True)
print('Accuracy of the network on the test images: {}% - FP32'.format(score))

Accuracy of the network on the test images: 98.48% - FP32


###  Квантование после обучения - Post-training quantization PTQ


Определим новую архитектуру квантованной сети, где мы также определяем заглушки квантования и деквантования, которые будут важны в начале и в конце.

Далее мы «объединим модули»; это может сделать модель быстрее за счет более оптимального распределения вычислений в памяти. 

In [16]:
qnet = Net(q=True)
load_model(qnet, net)
fuse_modules(qnet)

In [17]:
print_size_of_model(qnet)
score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused network on the test images: {}% - FP32'.format(score))

Size (MB): 0.179249
Accuracy of the fused network on the test images: 98.48% - FP32


Post-training static квантование   включает в себя не только преобразование весов из float в int, как при динамическом квантовании, но и выполнение дополнительного шага калибровки.


In [18]:
qnet.qconfig = torch.quantization.default_qconfig
print(qnet.qconfig)
torch.quantization.prepare(qnet, inplace=True)
print('Post Training Quantization Prepare: Inserting Observers')
print('\n Conv1: After observer insertion \n\n', qnet.conv1)

# Важный этап - калибровка модели
test(qnet, trainloader, cuda=False)


print('Post Training Quantization: Calibration done')
torch.quantization.convert(qnet, inplace=True)
print('Post Training Quantization: Convert done')
print('\n Conv1: After fusion and quantization \n\n', qnet.conv1)
print("Size of model after quantization")
print_size_of_model(qnet)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})
Post Training Quantization Prepare: Inserting Observers

 Conv1: After observer insertion 

 ConvReLU2d(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (1): ReLU()
  (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

 Conv1: After fusion and quantization 

 QuantizedConvReLU2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.055810220539569855, zero_point=0, bias=False)
Size of model after quantization
Size (MB): 0.050084


In [ ]:
score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused and quantized network on the test images: {}% - INT8'.format(score))

Accuracy of the fused and quantized network on the test images: 98.67% - INT8


Мы также можем определить пользовательскую конфигурацию квантования, в которой мы заменяем наблюдателей (observers) по умолчанию и вместо квантования по максимуму/минимуму можем взять среднее значение наблюдаемых значений, что, как мы надеемся, обеспечит лучшее качество.

In [19]:
from torch.quantization.observer import MovingAverageMinMaxObserver

qnet = Net(q=True)
load_model(qnet, net)
fuse_modules(qnet)

qnet.qconfig = torch.quantization.QConfig(
                                      activation=MovingAverageMinMaxObserver.with_args(reduce_range=True),
                                      weight=MovingAverageMinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))
print(qnet.qconfig)
torch.quantization.prepare(qnet, inplace=True)
print('Post Training Quantization Prepare: Inserting Observers')
print('\n Conv1: After observer insertion \n\n', qnet.conv1)

# Важный этап - калибровка модели
test(qnet, trainloader, cuda=False)

print('Post Training Quantization: Calibration done')
torch.quantization.convert(qnet, inplace=True)
print('Post Training Quantization: Convert done')
print('\n Conv1: After fusion and quantization \n\n', qnet.conv1)
print("Size of model after quantization")
print_size_of_model(qnet)
score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused and quantized network on the test images: {}% - INT8'.format(score))

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})
Post Training Quantization Prepare: Inserting Observers

 Conv1: After observer insertion 

 ConvReLU2d(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (1): ReLU()
  (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
)


/opt/conda/lib/python3.10/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Post Training Quantization: Calibration done
Post Training Quantization: Convert done

 Conv1: After fusion and quantization 

 QuantizedConvReLU2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.055435553193092346, zero_point=0, bias=False)
Size of model after quantization
Size (MB): 0.050084
Accuracy of the fused and quantized network on the test images: 98.45% - INT8


Кроме того, мы можем значительно улучшить точность, просто используя другую конфигурацию квантования. Мы повторяем то же упражнение с рекомендуемой конфигурацией для квантования для архитектур x86. Эта конфигурация делает следующее:
Квантует веса на основе каждого канала. Она
использует наблюдателя гистограммы, который собирает гистограмму активаций, а затем выбирает параметры квантования оптимальным образом.

In [ ]:
qnet = Net(q=True)
load_model(qnet, net)
fuse_modules(qnet)

In [22]:
qnet.qconfig = torch.quantization.get_default_qconfig('x86')
print(qnet.qconfig)

torch.quantization.prepare(qnet, inplace=True)
test(qnet, trainloader, cuda=False)
torch.quantization.convert(qnet, inplace=True)
print("Size of model after quantization")
print_size_of_model(qnet)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})
Size of model after quantization
Size (MB): 0.050084


In [23]:
score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused and quantized network on the test images: {}% - INT8'.format(score))

Accuracy of the fused and quantized network on the test images: 98.45% - INT8


### Обучение с учетом квантования

Обучение с учетом квантования (QAT) — это метод квантования, который обычно обеспечивает наивысшую точность. При QAT все веса и активации «поддельно квантуются» во время как прямого, так и обратного проходов обучения: то есть значения float округляются для имитации значений int8, но все вычисления по-прежнему выполняются с числами с плавающей точкой.

In [24]:
qnet = Net(q=True)
fuse_modules(qnet)
qnet.qconfig = torch.quantization.get_default_qat_qconfig('x86')

torch.quantization.prepare_qat(qnet, inplace=True)

print('\n Conv1: After fusion and quantization \n\n', qnet.conv1)
qnet=qnet.cuda()


 Conv1: After fusion and quantization 

 ConvReLU2d(
  1, 6, kernel_size=(5, 5), stride=(1, 1), bias=False
  (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
    (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
  )
  (activation_post_process): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
)


In [25]:
train(qnet, trainloader, cuda=True)

[1,     1]  loss 2.305163 (2.305163) train_acc 9.375000 (9.375000)
[1,   101]  loss 2.300375 (2.299719) train_acc 6.250000 (11.711015)
[1,   201]  loss 2.289497 (2.295191) train_acc 23.437500 (15.010883)
[1,   301]  loss 2.269218 (2.290190) train_acc 31.250000 (18.075166)
[1,   401]  loss 2.248153 (2.283823) train_acc 54.687500 (22.143859)
[1,   501]  loss 2.222377 (2.274365) train_acc 53.125000 (26.721557)
[1,   601]  loss 2.166247 (2.260955) train_acc 53.125000 (30.953619)
[1,   701]  loss 2.058138 (2.240316) train_acc 73.437500 (36.031116)
[1,   801]  loss 1.867005 (2.208219) train_acc 78.125000 (40.802512)
[1,   901]  loss 1.680785 (2.156148) train_acc 68.750000 (44.889359)
[2,     1]  loss 1.385889 (1.385889) train_acc 81.250000 (81.250000)
[2,   101]  loss 1.021710 (1.202953) train_acc 73.437500 (80.538366)
[2,   201]  loss 0.664465 (1.006807) train_acc 79.687500 (81.980721)
[2,   301]  loss 0.481810 (0.858740) train_acc 84.375000 (83.165490)
[2,   401]  loss 0.324097 (0.758068) 

In [26]:
qnet = qnet.cpu()
torch.quantization.convert(qnet, inplace=True)
print("Size of model after quantization")
print_size_of_model(qnet)

score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused and quantized network (trained quantized) on the test images: {}% - INT8'.format(score))

Size of model after quantization
Size (MB): 0.05572
Accuracy of the fused and quantized network (trained quantized) on the test images: 98.64% - INT8


Обучение квантованной модели с высокой точностью требует точного моделирования чисел при выводе. Поэтому для обучения с учетом квантования мы можем изменить цикл обучения, заморозив параметры квантизатора (масштаб и нулевая точка) и точно настроить веса.

In [ ]:
qnet = Net(q=True)

fuse_modules(qnet)

qnet.qconfig = torch.quantization.get_default_qat_qconfig('x86')
torch.quantization.prepare_qat(qnet, inplace=True)
qnet = qnet.cuda()


train(qnet, trainloader, cuda=True, q=True)

qnet = qnet.cpu()
torch.quantization.convert(qnet, inplace=True)

print("Size of model after quantization")
print_size_of_model(qnet)

score = test(qnet, testloader, cuda=False)
print('Accuracy of the fused and quantized network (trained quantized) on the test images: {}% - INT8'.format(score))

[1,     1]  loss 2.302550 (2.302550) train_acc 7.812500 (7.812500)
[1,   101]  loss 2.297554 (2.300715) train_acc 20.312500 (13.845916)
[1,   201]  loss 2.282641 (2.297055) train_acc 34.375000 (18.135883)
[1,   301]  loss 2.270876 (2.292123) train_acc 39.062500 (22.809385)
[1,   401]  loss 2.262033 (2.285715) train_acc 37.500000 (26.683292)
[1,   501]  loss 2.202892 (2.275584) train_acc 53.125000 (30.417290)
[1,   601]  loss 2.071962 (2.256968) train_acc 45.312500 (33.236273)
[1,   701]  loss 1.763640 (2.214525) train_acc 59.375000 (35.529601)
[1,   801]  loss 1.114516 (2.123895) train_acc 76.562500 (39.054697)
[1,   901]  loss 0.700801 (1.992948) train_acc 85.937500 (43.215871)
[2,     1]  loss 0.738226 (0.738226) train_acc 84.375000 (84.375000)
[2,   101]  loss 0.517096 (0.564659) train_acc 85.937500 (84.730817)
[2,   201]  loss 0.456042 (0.522940) train_acc 87.500000 (85.432214)
[2,   301]  loss 0.316576 (0.485950) train_acc 89.062500 (86.399502)
[2,   401]  loss 0.328412 (0.451746)